In [1]:
from work_2.helper import *
import tensorflow as tf 
import numpy as np 
from work_2.model2 import *
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from tqdm import tnrange,tqdm_notebook

In [2]:
epochs = 50
batch_size = 8
logs_path = "tensorflow_logs/fish2"
model_path = "models/t1"

In [3]:
imglist = img_location_reader("data")
train,test = train_test_split(imglist,test_size=0.3,random_state=0)
train,valid = train_test_split(train,test_size=0.1,random_state=0)

In [4]:
x_dev = img_location_list(train,over_sample=True)
print(len(x_dev))

9224


In [5]:
img_dummy_label = pd.get_dummies(list(image_label(x_dev).keys()))

In [6]:
x_val,y_val = val_test_image_reader(valid,size=(256,256),normalize = True)
# x_test,y_test = val_test_image_reader(test,size=(256,256))

100%|██████████| 274/274 [00:11<00:00, 24.34it/s]


In [8]:
print (x_val.shape)

(274, 256, 256, 3)


In [9]:
X = tf.placeholder(tf.float32,[None,256,256,3],name = "Input_data")
Y = tf.placeholder(tf.float32,[None,8], name = "InputLabels")

In [10]:

weights = {
"w1": init_weights([3,3,3,16]),
"w2": init_weights([3,3,16,16]),
"w3": init_weights([3,3,16,32]),
"w4": init_weights([3,3,32,32]),
"w5": init_weights([3,3,32,64]),
"w6": init_weights([3,3,64,64]),
"w7": init_weights([3,3,64,128]),
"w8": init_weights([3,3,128,128]),
"w9": init_weights([3,3,128,192]),
"w10": init_weights([3,3,192,192]),
"w11": init_weights([3,3,192,256]),
"w12": init_weights([3,3,256,256]),
"w13": init_weights([3,3,256,384]),
"w14": init_weights([384,1024]),
"w15": init_weights([1024,1024]),
"w16": init_weights([1024,8])
}

bias = {
"b1":init_bias([16]),
"b2":init_bias([16]),
"b3":init_bias([32]),
"b4":init_bias([32]),
"b5":init_bias([64]),
"b6":init_bias([64]),
"b7":init_bias([128]),
"b8":init_bias([128]),
"b9":init_bias([192]),
"b10":init_bias([192]),
"b11":init_bias([256]),
"b12":init_bias([256]),
"b13":init_bias([384]),
"b14":init_bias([1024]),
"b15":init_bias([1024])
}

In [11]:
pred = model(X,weights,bias)

In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,Y))
#optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.001,decay=0.9,momentum=0.9,epsilon=1e-10).minimize(cost)

In [13]:
#Evaluate the model
correct_pred= tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#Initialize all the variables
init = tf.initialize_all_variables()

saver = tf.train.Saver()

# Create summary to monitor cost tensor , accuracy tensor
tf.scalar_summary("loss",cost)
tf.scalar_summary("accuracy",accuracy)

merged_summary_op = tf.merge_all_summaries()

In [14]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.train.SummaryWriter(logs_path,graph=tf.get_default_graph())
    val_acc = []
    for ep in tqdm_notebook(range(epochs),desc="1st loop"):
        img_list = shuffle(x_dev,random_state=0)
        for i in tqdm_notebook(range(int(len(img_list)/batch_size)),desc="2nd loop",leave=False):
            img_loc = img_list[i*batch_size:(i+1)*batch_size]
            X_image, Y_image = dev_image_reader(img_loc,img_dummy_label,size = (256,256),normalize = True)
            _,c,summary = sess.run([optimizer,cost,merged_summary_op],feed_dict={X : np.float32(X_image), Y : np.float32(Y_image)})
            train_loss, train_acc = sess.run([cost,accuracy], feed_dict={X : np.float32(X_image), Y : np.float32(Y_image)})
            if i % 100 == 0:
                x = []
                for j in tqdm_notebook(range(len(x_val)),desc="3rd loop",leave=False):
                    xx_val = x_val[j][np.newaxis,:,:,:]
                    value = sess.run([tf.nn.softmax(pred)], feed_dict={X : np.float32(xx_val)})
                    x.append(value[0])
                x = np.concatenate(x)
                valid_acc = sess.run([tf.reduce_mean(tf.cast(tf.equal(tf.argmax(x,1),tf.argmax(Y,1)),tf.float32))],\
                        feed_dict={Y : np.float32(y_val)})
                print ("Train loss ="+"{:.6f}".format(train_loss),\
                       "Train acc ="+"{:.6f}".format(train_acc),\
                      "valid acc ="+"{:.6f}".format(valid_acc[0]))
                val_acc.append(valid_acc[0])
                if max(val_acc) == valid_acc[0]:
                    saver.save(sess,os.getcwd()+model_path+"_"+str(ep)+"_"+str(i))
        print ("epoch: ", ep)
    print("Optimization Finished!")

Train loss =2.079437 Train acc =0.375000 valid acc =0.463504
Train loss =2.078850 Train acc =0.125000 valid acc =0.014599
Train loss =13.001286 Train acc =0.250000 valid acc =0.463504
Train loss =2.107880 Train acc =0.000000 valid acc =0.051095
Train loss =2.121645 Train acc =0.000000 valid acc =0.080292
Train loss =2.068198 Train acc =0.250000 valid acc =0.080292
Train loss =2.079696 Train acc =0.000000 valid acc =0.040146
Train loss =2.080754 Train acc =0.000000 valid acc =0.197080
Train loss =2.079442 Train acc =0.000000 valid acc =0.197080
Train loss =2.079442 Train acc =0.500000 valid acc =0.197080
Train loss =2.079442 Train acc =0.125000 valid acc =0.197080
Train loss =2.079442 Train acc =0.125000 valid acc =0.197080
epoch:  0
Train loss =2.079442 Train acc =0.250000 valid acc =0.197080
Train loss =2.079442 Train acc =0.250000 valid acc =0.197080
Train loss =2.079442 Train acc =0.000000 valid acc =0.197080
Train loss =2.079442 Train acc =0.125000 valid acc =0.197080
Train loss =2

KeyboardInterrupt: 

In [14]:
# with tf.Session() as sess:
#     sess.run(init)
#     x =[]
#     for i in tqdm(range(len(x_val))):
#         xx_val = x_val[i][np.newaxis,:,:,:]
#         value = sess.run([tf.nn.softmax(pred)], feed_dict={X : np.float32(xx_val)})
#         x.append(value[0])
#     x = np.concatenate(x)
#     acc = sess.run([tf.reduce_mean(tf.cast(tf.equal(tf.argmax(x,1),tf.argmax(Y,1)),tf.float32))],\
#                         feed_dict={Y : np.float32(y_val)})
#     print (acc)